In [1]:
import os, warnings
import wandb

import pandas as pd
from fastai.vision.all import *
from sklearn.model_selection import StratifiedGroupKFold

import params

warnings.filterwarnings("ignore")

In [2]:
run = wandb.init(
    project=params.WANDB_PROJECT, entity=params.ENTITY, job_type="data_split"
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: reynald-havard. Use `wandb login --relogin` to force relogin


In [3]:
raw_data_at = run.use_artifact(f"{params.RAW_DATA_AT}:latest")
path = Path(raw_data_at.download())

wandb: Downloading large artifact bdd_simple_1k:latest, 846.57MB. 4007 files... 
wandb:   4007 of 4007 files downloaded.  
Done. 0:0:23.1


In [6]:
path

Path('/home/reynald/wandb-mlops-001/artifacts/bdd_simple_1k:v0')

In [4]:
path.ls()

(#5) [Path('/home/reynald/wandb-mlops-001/artifacts/bdd_simple_1k:v0/eda_table.table.json'),Path('/home/reynald/wandb-mlops-001/artifacts/bdd_simple_1k:v0/media'),Path('/home/reynald/wandb-mlops-001/artifacts/bdd_simple_1k:v0/images'),Path('/home/reynald/wandb-mlops-001/artifacts/bdd_simple_1k:v0/LICENSE.txt'),Path('/home/reynald/wandb-mlops-001/artifacts/bdd_simple_1k:v0/labels')]

In [5]:
fnames = os.listdir(path / "images")
groups = [s.split("-")[0] for s in fnames]

In [8]:
orig_eda_table = raw_data_at.get("eda_table")

wandb: Downloading large artifact bdd_simple_1k:latest, 846.57MB. 4007 files... 
wandb:   4007 of 4007 files downloaded.  
Done. 0:0:13.1


In [9]:
y = orig_eda_table.get_column("bicycle")

In [10]:
df = pd.DataFrame()
df["File_Name"] = fnames
df["fold"] = -1

In [11]:
cv = StratifiedGroupKFold(n_splits=10)
for i, (train_idxs, test_idxs) in enumerate(cv.split(fnames, y, groups)):
    df.loc[test_idxs, ["fold"]] = i

In [12]:
df["Stage"] = "train"
df.loc[df.fold == 0, ["Stage"]] = "test"
df.loc[df.fold == 1, ["Stage"]] = "valid"
del df["fold"]
df.Stage.value_counts()

Stage
train    800
test     100
valid    100
Name: count, dtype: int64

In [13]:
df.to_csv("data_split.csv", index=False)

In [14]:
processed_data_at = wandb.Artifact(params.PROCESSED_DATA_AT, type="split_data")

In [15]:
processed_data_at.add_file("data_split.csv")
processed_data_at.add_dir(path)

wandb: Adding directory to artifact (/home/reynald/wandb-mlops-001/artifacts/bdd_simple_1k:v0)... Done. 19.4s


In [16]:
data_split_table = wandb.Table(dataframe=df[["File_Name", "Stage"]])

In [17]:
join_table = wandb.JoinedTable(orig_eda_table, data_split_table, "File_Name")

In [18]:
processed_data_at.add(join_table, "eda_table_data_split")

ArtifactManifestEntry(path='eda_table_data_split.joined-table.json', digest='x3jnk6uRQOnBzGeD3OlkQA==', size=127, local_path='/home/reynald/.local/share/wandb/artifacts/staging/tmpx4ljcxch', skip_cache=False)

In [19]:
run.log_artifact(processed_data_at)
run.finish()